In [1]:
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras import metrics, regularizers
from keras.preprocessing import sequence
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [2]:
#Load cleaned dataset
data = pd.read_csv('../../Results/Cleaned_JobDescs.csv', header = 0, names = ['Query', 'Description'])
#data = pd.read_csv('../../Results/Cleaned_JobsNonIT.csv', header = 0, names = ['Query', 'Description'])

In [3]:
#Split the dataset to Training and Test subsets (90/10)
train, test = train_test_split(data, test_size = 0.1, random_state = 17) #random_state = None

train_descs = train['Description']
train_labels = train['Query']
 
test_descs = test['Description']
test_labels = test['Query']

In [8]:
# Model Parameters
vocab_size = 2000

sequences_length = 1200

embedding_dimensionality = 64 #possibly low??
max_features = 2000 #equal to vocab_size

num_labels = len(train_labels.unique())
batch_size = 32
nb_epoch = 20

nof_filters = 200 #check + research ... random now
kernel_size = 16 #check + research ... random now

hidden_dims = 1024

In [9]:
# Convert Texts to Numeric Vectors for Input
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_descs)

x_train = tokenizer.texts_to_sequences(train_descs)
x_test = tokenizer.texts_to_sequences(test_descs)

x_train = sequence.pad_sequences(x_train, maxlen = sequences_length, padding = 'post')
x_test = sequence.pad_sequences(x_test, maxlen = sequences_length, padding = 'post')

encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [10]:
model = Sequential()
model.add(Embedding(max_features, embedding_dimensionality, input_length = 1200))
#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

model.add(Conv1D(nof_filters, kernel_size, padding='valid', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', #'sgd', 'adam', 'RMSprop', 'Adagrad'
                   metrics = [metrics.categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1200, 64)          128000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1185, 200)         205000    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              205824    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
activation_3 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 25)                25625     
__________

In [11]:
history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = True,
                    validation_split = 0.2)

Train on 7200 samples, validate on 1800 samples
Epoch 1/20
7200/7200 [==============================] - 189s 26ms/step - loss: 2.9082 - categorical_accuracy: 0.1293 - val_loss: 2.2397 - val_categorical_accuracy: 0.2750
Epoch 2/20
7200/7200 [==============================] - 219s 30ms/step - loss: 1.7917 - categorical_accuracy: 0.4361 - val_loss: 1.4972 - val_categorical_accuracy: 0.5378
Epoch 3/20
7200/7200 [==============================] - 217s 30ms/step - loss: 1.1039 - categorical_accuracy: 0.6501 - val_loss: 1.2234 - val_categorical_accuracy: 0.6200
Epoch 4/20
7200/7200 [==============================] - 232s 32ms/step - loss: 0.6795 - categorical_accuracy: 0.7846 - val_loss: 1.2003 - val_categorical_accuracy: 0.6339
Epoch 5/20
7200/7200 [==============================] - 261s 36ms/step - loss: 0.3902 - categorical_accuracy: 0.8856 - val_loss: 1.3220 - val_categorical_accuracy: 0.6328
Epoch 6/20
7200/7200 [==============================] - 254s 35ms/step - loss: 0.2275 - categoric

In [12]:
score = model.evaluate(x_test, y_test, batch_size = batch_size, verbose = True)
 
print('\nTest categorical_crossentropy:', score[0])
print('Categorical accuracy:', score[1])


1000/1000 [==============================] - 9s 9ms/step

Test categorical_crossentropy: 1.7260095990896225
Categorical accuracy: 0.651
